### Import Required Libraries and Set Up Environment Variables

In [20]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [21]:
# Set environment variables from the .env in the local environment
load_dotenv('example.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [22]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# No thanks, I'll build PARAMETERS
parameters = {
    'begin_date':begin_date,
    'end_date':end_date,
    'fq': filter_query,
    'fl': field_list,
    'sort': sort,
    'page':0, # this will increment in the for loop
    'api-key': nyt_api_key
}


In [23]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):

    # create query with a page number
    parameters['page'] = page
    # API results show 10 articles at a time

    # Make a "GET" request and retrieve the JSON
    nyt_response = requests.get(url, params=parameters)
    reviews = nyt_response.json()
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        for review in reviews["response"]["docs"]:
        # loop through the reviews["response"]["docs"] and append each review to the list
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f'Checked page {page}: {len(reviews["response"]["docs"])} articles')

        # Print the page number that had no results then break from the loop
    except:
        print(f'{page} had no results')
        break

Checked page 0: 10 articles
Checked page 1: 10 articles
Checked page 2: 10 articles
Checked page 3: 10 articles
Checked page 4: 10 articles
Checked page 5: 10 articles
Checked page 6: 10 articles
Checked page 7: 10 articles
Checked page 8: 10 articles
Checked page 9: 10 articles
Checked page 10: 10 articles
Checked page 11: 10 articles
Checked page 12: 10 articles
Checked page 13: 10 articles
Checked page 14: 10 articles
Checked page 15: 10 articles
Checked page 16: 10 articles
Checked page 17: 10 articles
Checked page 18: 10 articles
Checked page 19: 10 articles


In [24]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [25]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [26]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

import re

pattern_1 = re.compile(r" ?\u2018(.*)\u2019 ?")
pattern_2 = re.compile(r"^(.*?)(?:’ .+)?$")
pattern_3 = re.compile(r"^(.*?),?$")


reviews_df['title'] = reviews_df['headline.main'].str.extract(pattern_1)
reviews_df['title'] = reviews_df['title'].str.extract(pattern_2)
reviews_df['title'] = reviews_df['title'].str.extract(pattern_3)
reviews_df[['headline.main', 'title']]

,headline.main,title
0,"‘The Attachment Diaries’ Review: Love, Sick",The Attachment Diaries
1,Review: ‘What’s Love Got to Do With It?’ Proba...,What’s Love Got to Do With It?
2,‘You Can Live Forever’ Review: Do You Love Me ...,You Can Live Forever
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,A Tourist’s Guide to Love
4,‘Other People’s Children’ Review: True Romance,Other People’s Children
...,...,...
195,Review: A Combustible Pair Find Love in ‘The O...,The Other Half
196,"Review: Love as the World Wars, in ‘The Ottoma...",The Ottoman Lieutenant
197,Review: It’s All Mirth and Taxes in ‘Love & Ta...,Love & Taxes
198,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",Everybody Loves Somebody


In [27]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Everybody Loves Somebody


In [28]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
movie_titles = reviews_df['title'].to_list()

# There was a NaN value in the list (a byproduct of trying to extract a title from a review that mentioned no title)
# TMDB will return data on a movie called "Nan", so I'm removing it from the list to avoid that unintended behavior
movie_titles = [x for x in movie_titles if not pd.isna(x)]
movie_titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journa

### Access The Movie Database API

In [29]:
# Prepare The Movie Database query
tmdb_url = "https://api.themoviedb.org/3/search/movie"

tmdb_parameters = {
    'query': '', # Replaced with a movie title in the for loop
    'api_key': tmdb_api_key
}

In [30]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in movie_titles:
    # Check if we need to sleep before making a request
    if request_counter in [50, 100, 150, 200]:
        print(f'Sleeping at {request_counter} requests')
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    tmdb_parameters['query'] = title
    tmdb_response = requests.get(url = tmdb_url, params = tmdb_parameters)
    tmdb_json = tmdb_response.json()
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        movie = tmdb_json['results'][0]

        # Get movie id
        id = movie['id']

        # Make a request for a the full movie details
        full_details_response = requests.get(url=f'https://api.themoviedb.org/3/movie/{id}?api_key=6d0bcaa7aefccc749d7286ac77591f8b')
        full_details_json = full_details_response.json()
        # Execute "GET" request with url

        # Extract the genre names into a list
        genres = [genre['name'] for genre in full_details_json['genres']]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang['english_name'] for lang in full_details_json['spoken_languages']]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in full_details_json['production_countries']]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_dict = {
            'title': title,
            'original_title': full_details_json['original_title'],
            'budget': full_details_json['budget'],
            'genres': genres,
            'language': full_details_json['original_language'],
            'spoken_languages':spoken_languages,
            'homepage': full_details_json['homepage'],
            'overview': full_details_json['overview'],
            'popularity': full_details_json['popularity'],
            'runtime': full_details_json['runtime'],
            'revenue': full_details_json['revenue'],
            'release_date': full_details_json['release_date'],
            'vote_avergae': full_details_json['vote_average'],
            'vote_count': full_details_json['vote_count'],
            'production_countries': production_countries
        }

        tmdb_movies_list.append(tmdb_dict)

        # Print out the title that was found
        print(f'"{title}" found.')
    except:
        print(f'"{title}" not found.')


"The Attachment Diaries" found.
"What’s Love Got to Do With It?" found.
"You Can Live Forever" found.
"A Tourist’s Guide to Love" found.
"Other People’s Children" found.
"One True Loves" found.
"The Lost Weekend: A Love Story" found.
"A Thousand and One" found.
"Your Place or Mine" found.
"Love in the Time of Fentanyl" found.
"Pamela, a Love Story" found.
"In From the Side" found.
"After Love" found.
"Alcarràs" found.
"Nelly & Nadine" found.
"Lady Chatterley’s Lover" found.
"The Sound of Christmas" found.
"The Inspection" found.
"Bones and All" found.
"My Policeman" found.
"About Fate" found.
"Waiting for Bojangles" found.
"I Love My Dad" found.
"A Love Song" found.
"Alone Together" found.
"Art of Love" found.
"The Wheel" found.
"Thor: Love and Thunder" found.
"Both Sides of the Blade" found.
"Fire of Love" found.
"Love & Gelato" found.
"Stay Prayed Up" found.
"Benediction" found.
"Dinner in America" found.
"In a New York Minute" found.
"Anaïs in Love" found.
"I Love America" found.
"S

In [31]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "language": "es",
        "spoken_languages": [
            "Spanish"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 2.24,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_avergae": 3.0,
        "vote_count": 4,
        "production_countries": [
            "Argentina"
        ]
    },
    

In [32]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_avergae,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",2.240,102,0,2021-10-07,3.000,4,[Argentina]
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,"[Romance, Comedy]",en,"[English, Portuguese, Urdu]",,Two childhood friends now in their thirties mu...,25.399,109,10898395,2023-01-26,6.074,175,"[France, United Kingdom]"
2,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.437,96,15055,2023-03-24,6.600,37,"[Canada, United States of America]"
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.249,96,0,2023-04-21,6.287,162,[United States of America]
4,Other People’s Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[French, English]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",15.277,104,84178,2022-09-21,6.818,190,[France]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,The Other Half,The Other Half,0,"[Drama, Romance]",en,[English],http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,5.348,103,0,2016-12-02,6.300,24,[Canada]
194,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[Turkish, English]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",30.052,111,413844,2017-03-10,6.164,247,"[Turkey, United States of America]"
195,Love & Taxes,Love & Taxes,0,[Comedy],en,[English],http://loveandtaxesmovie.com,Love &amp; Taxes is a riveting comic tale of s...,2.566,98,0,2017-03-03,4.000,1,[United States of America]
196,Everybody Loves Somebody,Everybody Loves Somebody,2000000,"[Romance, Comedy]",en,"[English, Spanish]",,"On the surface, Clara Barron seems to have it ...",15.971,102,0,2017-02-10,6.538,159,[Mexico]


### Merge and Clean the Data for Export

In [33]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged = pd.merge(tmdb_df, reviews_df, on='title', how='left')
merged

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",2.240,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,"[Romance, Comedy]",en,"[English, Portuguese, Urdu]",,Two childhood friends now in their thirties mu...,25.399,109,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.437,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.249,96,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,Other People’s Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[French, English]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",15.277,104,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,The Other Half,The Other Half,0,"[Drama, Romance]",en,[English],http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,5.348,103,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
198,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[Turkish, English]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",30.052,111,...,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
199,Love & Taxes,Love & Taxes,0,[Comedy],en,[English],http://loveandtaxesmovie.com,Love &amp; Taxes is a riveting comic tale of s...,2.566,98,...,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
200,Everybody Loves Somebody,Everybody Loves Somebody,2000000,"[Romance, Comedy]",en,"[English, Spanish]",,"On the surface, Clara Barron seems to have it ...",15.971,102,...,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [34]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for col in to_fix:
    # Convert the column to type 'str'
    merged[col] = merged[col].astype(str)

    # Loop through characters to remove
    for char in to_remove:
        merged[col] = merged[col].str.replace(char, '')

# Display the fixed DataFrame
merged.head()

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",2.240,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,"Romance, Comedy",en,"English, Portuguese, Urdu",,Two childhood friends now in their thirties mu...,25.399,109,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.437,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.249,96,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,Other People’s Children,Les Enfants des autres,0,"Drama, Comedy",fr,"French, English",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",15.277,104,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [35]:
# Drop "byline.person" column
merged = merged.drop('byline.person', axis=1)

In [36]:
# Delete duplicate rows and reset index
merged = merged.drop_duplicates('title').reset_index(drop=True)
merged

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",2.240,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,What’s Love Got to Do With It?,What's Love Got to Do with It?,0,"Romance, Comedy",en,"English, Portuguese, Urdu",,Two childhood friends now in their thirties mu...,25.399,109,...,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.437,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
3,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.249,96,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
4,Other People’s Children,Les Enfants des autres,0,"Drama, Comedy",fr,"French, English",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",15.277,104,...,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,The Other Half,The Other Half,0,"Drama, Romance",en,English,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,5.348,103,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None
192,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"Romance, Drama, War",en,"Turkish, English",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",30.052,111,...,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,None
193,Love & Taxes,Love & Taxes,0,Comedy,en,English,http://loveandtaxesmovie.com,Love &amp; Taxes is a riveting comic tale of s...,2.566,98,...,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,None
194,Everybody Loves Somebody,Everybody Loves Somebody,2000000,"Romance, Comedy",en,"English, Spanish",,"On the surface, Clara Barron seems to have it ...",15.971,102,...,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,None


In [37]:
# Export data to CSV without the index
merged.to_csv('final_csv.csv', index=False)